Based on this notebook:   
https://www.kaggle.com/abhilashawasthi/not-so-fancy-but-fast-benchmark  
Including changes from this notebook:   
https://www.kaggle.com/hengzheng/time-is-our-best-friend  
  
Rewritten to be more pythonic and readable (and hence, extendable).  
29 lines of code!!!

Updated: for customer, use season's data, but for popularity, use only more recent.

In [ ]:
import pandas as pd

data_path = '/kaggle/input/h-and-m-personalized-fashion-recommendations/'
t_file, sub_file = 'transactions_train.csv', 'sample_submission.csv'

# set dtype or pandas will drop the leading '0' and convert to int
t = pd.read_csv(data_path + "/" + t_file, dtype={'article_id': str})
sub = pd.read_csv(data_path + "/" + sub_file)

t_season = t[t['t_dat'] > '2020-08-01'].copy()
t_new = t[t['t_dat'] > '2020-09-07'].copy()

# customer purchases (cp)
cp = t_season.groupby(["customer_id", "article_id"])[["article_id"]].count()
cp.columns = ["purchase_count"]
cp = cp.reset_index()
cp = cp.sort_values(["customer_id", "purchase_count"], ascending=False)
cp = cp.groupby("customer_id").head(10)
cp = cp.groupby("customer_id")["article_id"].agg(list)

# popular purchases (pp)
pp = list(t_new['article_id'].value_counts().index[:12])

# submission (sub)
sub["prediction"] = sub["customer_id"].map(cp)
sub["prediction"] = sub["prediction"].apply(lambda x: x if isinstance(x, list) else [])
sub["prediction"] = sub["prediction"].apply(lambda x: x[:12] + pp[:12-len(x)])
sub["prediction"] = sub["prediction"].apply(lambda x: " ".join(x))

sub.to_csv('simpler_submission.csv', index=False)